In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random 

In [223]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [224]:
data_video = np.load("final_features/video_features_and_labels.npz")
features_video = data_video["features"]
labels_video = data_video["labels"]
print(features_video.shape)

(309, 128)


In [225]:
data_transcription = np.load("final_features/transcription_features_and_labels.npz")
features_transcription = data_transcription["features"]
labels_transcription = data_transcription["labels"]
print(features_transcription.shape)

(309, 64)


In [226]:
data_audio = np.load("final_features/audio_features_and_labels.npz")
features_audio = data_audio["features"]
labels_audio = data_audio["labels"]
print(features_audio.shape)

(309, 20)


In [227]:
final_features = np.hstack((features_video, features_transcription, features_audio))


In [228]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
final_features = scaler.fit_transform(final_features)

In [229]:
dataset = TensorDataset(torch.tensor(final_features, dtype=torch.float32), torch.tensor(labels_video, dtype=torch.float32))
dataset_loader = DataLoader(dataset, batch_size=32, shuffle=False)

train_size = int(0.7 * len(dataset))  # 70% para entrenamiento
val_size = int(0.15 * len(dataset))   # 15% para validación
test_size = len(dataset) - train_size - val_size  # 15% para prueba

train, val, test = random_split(dataset, [train_size, val_size, test_size])

batch_size = 4
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [230]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
print(device)

cuda


In [231]:
def evaluar_modelo_1(model, test_loader,path):
    model.load_state_dict(torch.load(path, weights_only=True))
    model.to(device)  
    model.eval()  # Poner el modelo en modo evaluación

    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            output = model(x_batch)
            probs = torch.sigmoid(output)
            preds = (probs >= 0.5).float()  # Convertir en 0 o 1
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    print(f"Total ejemplos en test: {total}")
    acc = correct / total
    print(f"Precisión en prueba: {acc:.4f}")  

In [232]:
class MultimodalClassifier(nn.Module):
    def __init__(self, hidden_dim=512, input_dim=212):  
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.GELU(),
            nn.LayerNorm(256),
            nn.Dropout(0.3),


            
            nn.Linear(256, 128),
            nn.GELU(),
            nn.LayerNorm(128),
            nn.Dropout(0.3),

            nn.Linear(128, 1)  
        )

    def forward(self, x):
        return self.classifier(x)


In [233]:
# Crear el modelo y definir todo
model = MultimodalClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,
                              mode='max',      
                              patience=5,
                              factor=0.3,)

best_acc = 0.0  
best_model_path = "concat_final_model.pth"  

# Early Stopping
patience = 100
no_improve = 0

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)  
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            output = model(x_batch)
            probs = torch.sigmoid(output)
            preds = (probs >= 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    acc = correct / total
    scheduler.step(acc)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Val Acc: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Mejores pesos guardados con precisión: {best_acc:.4f}")
        no_improve = 0  # reinicia el contador
    else:
        no_improve += 1
        print(f"No mejora por {no_improve}/{patience} épocas")

    if no_improve >= patience:
        print(f"Early stopping activado en la época {epoch+1}")
        break


Epoch 1/1000, Loss: 0.2364, Val Acc: 0.7174
Mejores pesos guardados con precisión: 0.7174
Epoch 2/1000, Loss: 0.0493, Val Acc: 0.7174
No mejora por 1/100 épocas
Epoch 3/1000, Loss: 0.0263, Val Acc: 0.7174
No mejora por 2/100 épocas
Epoch 4/1000, Loss: 0.0213, Val Acc: 0.7174
No mejora por 3/100 épocas
Epoch 5/1000, Loss: 0.0131, Val Acc: 0.7174
No mejora por 4/100 épocas
Epoch 6/1000, Loss: 0.0118, Val Acc: 0.7391
Mejores pesos guardados con precisión: 0.7391
Epoch 7/1000, Loss: 0.0097, Val Acc: 0.7391
No mejora por 1/100 épocas
Epoch 8/1000, Loss: 0.0082, Val Acc: 0.7391
No mejora por 2/100 épocas
Epoch 9/1000, Loss: 0.0063, Val Acc: 0.7391
No mejora por 3/100 épocas
Epoch 10/1000, Loss: 0.0051, Val Acc: 0.7391
No mejora por 4/100 épocas
Epoch 11/1000, Loss: 0.0048, Val Acc: 0.7391
No mejora por 5/100 épocas
Epoch 12/1000, Loss: 0.0053, Val Acc: 0.7391
No mejora por 6/100 épocas
Epoch 13/1000, Loss: 0.0042, Val Acc: 0.7391
No mejora por 7/100 épocas
Epoch 14/1000, Loss: 0.0042, Val Ac

In [234]:
evaluar_modelo_1(model,test_loader, path="concat_final_model.pth")

Total ejemplos en test: 47
Precisión en prueba: 0.6809
